In [17]:
import pandas as pd
import numpy as np

# tf-idf
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

# word2vec
from gensim.models.word2vec import Word2Vec
from gensim.models.word2vec import LineSentence

# scatter plot
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# 解决matplotlib以及seaborn中文字方块的问题
from pylab import mpl
import seaborn as sns

# 3D plot
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

# k-means
from sklearn.cluster import KMeans
from sklearn import cluster
from sklearn import metrics

# 分词方程： seperater
import jieba
import re
import jieba.posseg as pseg

In [18]:
# icd9-cm3 导入
icd9v2 = pd.read_csv(r'~\Desktop\icd9_cm3\resource\icd9_cm3.csv',encoding = 'gb18030')
icd9v3 = icd9v2[icd9v2['detail_name'] != '(null)']
icd9v3 = icd9v3.reset_index(drop=True)
icd9v3['detail_name'].describe()

count      10216
unique      9658
top       耳硬化分离术
freq           3
Name: detail_name, dtype: object

In [19]:
icd9_guangzhou = pd.read_excel(r'C:\Users\MYTh_\Desktop\icd9_cm3\resource\广东省ICD-9-CM-3手术与操作分类代码-20180101.xls',encoding = 'gb18030')
icd9_beijing = pd.read_excel(r'C:\Users\MYTh_\Desktop\icd9_cm3\resource\北京版ICD－9－DM－3为基础的手术代码 Microsoft Excel 工作表.xls',encoding = 'gb18030')

In [20]:
icd9_guangzhou = icd9_guangzhou.rename(index=str, columns={"fopname": "detail_name"})
icd9_beijing = icd9_beijing.rename(index=str, columns={"描述": "detail_name"})

seg_df = icd9v3[['detail_name']]
seg_df = seg_df.append(icd9_guangzhou[["detail_name"]])
seg_df = seg_df.append(icd9_beijing[["detail_name"]])

seg_df = seg_df.drop_duplicates(keep = 'first')
seg_df = seg_df.reset_index(drop = True)

### 制作字典
- 目标：合并原先被错误分词的词语，去掉没有意义的高频次
- 资源来源
    - 北京大学云中心的“提交-字典数据”
    - [王晔](wangye@wondersgroup.com)提供的“组织、部位与治疗方式.xlsx与新词探查工具

In [22]:
# 已知字典导入
dirc = pd.read_csv(r'C:\Users\MYTh_\Desktop\icd9_cm3\resource\医学类词库.txt', engine = "python" ,encoding = 'utf-8', header = None)
body_part = pd.read_excel(r'C:\Users\MYTh_\Desktop\icd9_cm3\resource\组织、部位与治疗方式.xlsx',encoding = 'gb18030')
body_part2 = pd.read_excel(r'C:\Users\MYTh_\Desktop\icd9_cm3\resource\提交-词典数据.xlsx',encoding = 'gb18030',sheetname = '手术及治疗')
# 未知字典制作：通过正反向最大词向量在原数据中发现新词
fs = pd.read_csv(r'~\Desktop\icd9_cm3\resource\forward.csv',encoding = 'utf-8', header = [-1])
bs = pd.read_csv(r'~\Desktop\icd9_cm3\resource\backward.csv',encoding = 'utf-8', header = [-1])

C:\Users\MYTh_\Anaconda3\lib\site-packages\pandas\io\excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


In [23]:
fsbox = []

for word in fs[0]:
    temp = word.split(' ')
    for i in range(len(temp)):
#        if len(temp[i]) == 1 or len(temp[i]) > 6:   #只选取字符长度 < = 5 的术式名称
#            continue
#        else:
            fsbox.append(temp[i])

bsbox = []

for word in bs[0]:
    temp = word.split(' ')
    for i in range(len(temp)):
        bsbox.append(temp[i])     

In [24]:
#字典建立
dictionary = []

#导入第一本字典
for i in range(len(body_part.columns)):
    temp = body_part[body_part.columns[i]].dropna()    
    for j in range(len(temp)):
        dictionary.append(temp[j])

#导入第二本字典
for i in range(len(body_part2['词语'])):
    if len(str(body_part2['词语'][i])) <  5:
        dictionary.append(len(body_part2['词语'][i]))
    else:
        continue

# 导入正向查找字典
for word in fsbox:
    dictionary.append(word)
    
# 导入反向查找字典
for word in bsbox:
    dictionary.append(word)
    
# 导入医学字典
for word in dirc[0].tolist():
    dictionary.append(word)

np.savetxt(r'C:\Users\MYTh_\Desktop\wyfc_2015\dictionary.txt', dictionary, fmt='%s',encoding = 'utf-8')

In [25]:
# 不同结构的字典，为了后续的分析而建立
list_dic = []

for word in dictionary:
    list_dic.append(re.findall('[\u4E00-\u9FA5A-Za-z]+',str(word)))   

In [26]:
# 得到并保存去掉不需要字符且没有被分割的手术名称

for i in range(len(icd9v3.detail_name)):      
    hans = re.findall('[\u4E00-\u9FA5A-Za-z]+',icd9v3.detail_name[i])
    combine = ''.join(hans)
    icd9v3.loc[i, 'clean_dn'] = combine

np.savetxt(r'C:\Users\MYTh_\Desktop\wyfc_2015\test.txt', icd9v3.clean_dn, fmt='%s',encoding = 'utf-8')

In [27]:
for word in list_dic:
    jieba.suggest_freq(word, True)   #导入字典

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\MYTh_\AppData\Local\Temp\jieba.cache
Loading model cost 0.732 seconds.
Prefix dict has been built succesfully.


In [28]:
# 分词方程： seperater
def seperater(param):
        
    hans_before = "".join(re.findall('[\u4E00-\u9FA5A-Za-z]+',param))
    hans_after = re.sub(u"\,\'|(术+)|(的+)|(病+)","",hans_before)
    after_seg = ''.join(hans_after)
    
    seg_list = jieba.cut(after_seg, cut_all=False)
    temp = " ".join(seg_list)

    return temp;

In [29]:
# 执行分词并保存
for i in range(len(icd9v3)):
    icd9v3.loc[i, 'tokens'] = seperater(icd9v3.detail_name[i])

for i in range(len(seg_df)):
    seg_df.loc[i, 'tokens'] = seperater(seg_df.detail_name[i])
    


In [30]:
np.savetxt(r'C:\Users\MYTh_\Desktop\icd9_cm3\result\segment.txt', seg_df, fmt='%s',encoding = 'utf-8')   

### 特征词提取
一般情况下，名词和动词比其他词性的词重要。另外，词语包括的字数越多，包含的信息量越大。论文中定义了一种基于词性和词长度的特征词权重计算公式，即
                        $Weight{w_{id}} = \lambda Weight_{pos}(w_{id})+(1-\lambda)Weight_{len}(w_{id})$
式中
- $Weight(w_{id})$表示词语$w_i$在文本$d$中的权重
- $Weight_{pos}(w_{id})$表示$w_i$在文本d中的词性权重
- $Weight_{len}(w_{id})$表示$w_i$在文本d中的长度权重
- $\lambda$和$(1-\lambda)$为加权系数，$\lambda$取经验值0.6
- $Weight_{pos}(w_{id})$和$Weight_{len}(w_{id})$的具体计算公式为



In [14]:
# 权重分词 
posdf = pd.DataFrame()

import jieba.posseg as pseg

for word in list_dic:
    jieba.suggest_freq(word, True)

for i in range(len(icd9v3.clean_dn)):
    words = pseg.cut(icd9v3.clean_dn[i])
    for word1,j in zip(words,range(16)):
        posdf.loc[i,j] = '%s'%word1

### 词向量模型训练

In [56]:
# 词向量训练
model = Word2Vec(sentences = LineSentence(r'~\Desktop\icd9_cm3\result\segment.txt'),
                 size = 300, 
                 min_count = 2,
                 sg=1)  
model.save('op_model.model')

C:\Users\MYTh_\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [57]:
# 高频词查找
freq_count = []

for w in model.wv.vocab:
        freq_count.append([w,model.wv.vocab[w].count])
token_freq = pd.DataFrame(data = freq_count, columns = ['token_name','frequency'])
np.savetxt(r'C:\Users\MYTh_\Desktop\icd9_cm3\result\high_freq.txt', token_freq.sort_values(by = ["frequency"],ascending = False), fmt='%s',encoding = 'utf-8')
token_freq = token_freq.sort_values(by = ["frequency"],ascending = False)

In [17]:
hf_writer = pd.ExcelWriter(r'C:\Users\MYTh_\Desktop\icd9_cm3\high_freq.xlsx')
token_freq.sort_values(by = ["frequency"],ascending = False).to_excel(hf_writer,'Sheet1')
hf_writer.save()

In [18]:
# 训练范围：全部词
# 求出每个手术名称的词向量均值
vector_list = []

for i in range(len(icd9v3.tokens)):
    vector_list.append(np.mean(model[list(icd9v3.tokens[i].split(' '))], axis = 0))

C:\Users\MYTh_\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [26]:
# 训练范围：匹配词
# 求出每个手术名称的词向量均值
vector_list = []

for i in range(len(matched_tokens)):
    vector_list.append(np.mean(model[list(matched_tokens[i].split(' '))], axis = 0))

C:\Users\MYTh_\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [27]:
# k均值聚类
# 聚类数量：默认4000；如果需要测试匹配词需要调低
kmeans = cluster.KMeans(n_clusters = 4000 ,max_iter = 500)
kmeans.fit(vector_list)
labels= kmeans.predict(vector_list)

In [22]:
# 标记所有手术名称并保存
icd9v3["label"] = labels
icd9v3_sorted = icd9v3.sort_values(by = ['label'], ascending = (True))
labeled_icd9 = icd9v3_sorted[['detail_code','detail_name','label']]
writer = pd.ExcelWriter(r'C:\Users\MYTh_\Desktop\icd9_cm3\result\labeled_ICD9-CM3_test.xlsx')
labeled_icd9.to_excel(writer,'Sheet1')
writer.save()

### 测试代码
1. 匹配词分类查看
2. 匹配词计数

In [48]:
# 匹配词单独分类文件保存

matched = pd.DataFrame()
matched['sp_names'] = matched_tokens
matched["label"] = labels

matched = matched.sort_values(by = ['label'], ascending = (True))

for i in range(len(matched)):
    matched.loc[i,'detail_name'] = "".join(matched.sp_names[i].split(' '))

cleandf = pd.merge(matched, icd9v3, how = 'left' ,left_on = ['sp_names'], right_on = ['tokens'])
clean = cleandf[['detail_code','detail_name_x','label_y']]
clean = clean.sort_values(by = ['label_y'], ascending = (True))
clean = clean.drop_duplicates()
clean = clean.reset_index(drop = True)    
    
writer = pd.ExcelWriter(r'C:\Users\MYTh_\Desktop\icd9_cm3\result\match-labeled_ICD9-CM3.xlsx')
clean.to_excel(writer,'Sheet1')
writer.save()

In [25]:
# 匹配/不匹配词计数

matched_tokens = []
unmatched_tokens= []

for i in range(len(icd9v3.tokens)):
    temp = icd9v3.tokens[i].split(' ')
    for word in temp:
        if word in dictionary:
            matched_tokens.append(icd9v3.tokens[i])
            break
        else:
            unmatched_tokens.append(icd9v3.tokens[i])
            break  
            
print("匹配成功的分词有：" + str(len(matched_tokens)))
print("匹配失败的分词有：" + str(len(unmatched_tokens)))

匹配成功的分词有：8821
匹配失败的分词有：1395
